**5. Collect Keypoint Values for Training and Testing**

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt
import time

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model is used for pose detection
mp_drawing = mp.solutions.drawing_utils # Drawing utilities on the image detected by the model

: 

In [ ]:
def mediapipe_detection(image, model):

    # Convert the BGR image to RGB before processing
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR to RGB because the model is trained on RGB images

    # Image is no longer writeable
    image.flags.writeable= False  

    # Make prediction
    results = model.process(image) # Make prediction on the image using the model and store the results in results

    # Image is now writeable
    image.flags.writeable = True 

    # Convert back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # RGB to BGR because we are going to display the image using cv2
    return image, results # Return the image and the results. Results contain the landmarks of the image

: 

In [ ]:
def draw_styled_landmarks (image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, # Draw the face landmarks
    mp_drawing. DrawingSpec (color=(80,110,10), thickness=1, circle_radius=1), # Draw the face connections
    mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) # Draw the face connections
    )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic. POSE_CONNECTIONS, # Draw the pose landmarks
    mp_drawing. DrawingSpec (color=(80,22,10), thickness=2, circle_radius=4), # Draw the pose connections
    mp_drawing. DrawingSpec (color=(80,44,121), thickness=2, circle_radius=2) # Draw the pose connections
    )

    # Draw Left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic. HAND_CONNECTIONS, # Draw the left hand landmarks 
    mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), # Draw the left hand connections
    mp_drawing.DrawingSpec (color= (121,44,250), thickness=2, circle_radius=2) # Draw the left hand connections
    )

    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, # Draw the right hand landmarks
    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), # Draw the right hand connections
    mp_drawing. DrawingSpec (color=(245,66,230), thickness=2, circle_radius=2) # Draw the right hand connections
    )

: 

In [ ]:
def extract_keypoints(results): # Extract the landmarks from the results
    # Extract pose landmarks
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    # Extract face landmarks
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    # Extract left hand landmarks
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    # Extract right hand landmarks
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    # Concatenate the landmarks
    return np.concatenate([pose, face, lh, rh])

: 

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') # Create a folder named MP_Data in the current directory
# Actions that we try to detect
actions = np.array(['again','boy','deaf','finish','girl','go','goodbye','hearing','hello','help',
'how','i','know','like','love','man','more','my','need','no',
'none','not','please','right','school','science','sentence','sign-language','sorry','sports',
'student','thank-you','that','want','what','when','where','who','why','with',
'without','woman','yes','you','your','i-love-you','youre-welcome','intelligent','friend','mother',
'father','see-you-later','baby','name'])
no_sequences = 30  # Number of sequences for each action
sequence_length = 30  # Number of frames in each sequence

: 

In [ ]:
for action in actions: # For each action
    for sequence in range(no_sequences): # For each sequence
        try:
            os.makedirs(os.path. join(DATA_PATH, action, str(sequence))) # Create a folder for each sequence
        except:
            pass

: 

In [ ]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
#     for actions[0] in actions:
#         # Loop through sequences aka videos
#         for sequence in range(no_sequences):
#             # Loop through video length aka sequence length
#             for frame_num in range(sequence_length):

#                 # Read feed
#                 ret, frame = cap.read()

#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic)
#                 # print(results)

#                 # Draw landmarks
#                 draw_styled_landmarks(image, results)
                
#                 # NEW Apply wait logic
#                 if frame_num == 0: 
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(2000)
#                 else: 
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
                
#                 # NEW Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)

#                 # Break gracefully
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#     cap.release()
#     cv2.destroyAllWindows() 

: 

In [ ]:
# video capture
cap = cv2.VideoCapture(0)


#Action To Capture
acts = []
acts.append('what')


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:  # Initialize the holistic model
    
    # NEW LOOP
    # Loop through actions
    cv2.waitKey(10000) # Wait for 10 seconds
    for act in acts: # For each action
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read() # Read the feed i.e. the video frame

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results) # Draw the landmarks on the image
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (20,40), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA) # Display the text on the image about starting the collection
                    cv2.putText(image, '"{}" {}'.format(act, sequence), (570,40), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (50,205,50), 4, cv2.LINE_AA) # Display the text on the image about the action and sequence
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image) # Display the image
                    cv2.waitKey(2000) # Wait for 2 seconds
                else: 
                    cv2.putText(image, '"{}" {}'.format(act, sequence), (570,40), # Display the text on the image about the action and sequence
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA) 
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image) # Display the image  
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results) # Extract the keypoints from the results i.e. the landmarks
                npy_path = os.path.join(DATA_PATH, act, str(sequence), str(frame_num)) # Create a path to save the keypoints
                np.save(npy_path, keypoints) # Save the keypoints in the folder

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'): # If the user presses the 'q' key it will break the loop
                    break
                    
    cap.release() 
    cv2.destroyAllWindows()

: 